In [1]:
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers import concatenate
from keras.layers.core import Dense, Activation, Dropout, Flatten, Lambda 
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from scipy.ndimage.interpolation import rotate, shift, zoom
from keras.constraints import max_norm
from keras.layers import Dense, Dropout
from keras.layers import Flatten,  MaxPooling2D, Conv2D
from keras.callbacks import TensorBoard
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import json
from PIL import Image
import h5py
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from prediction_utils import predict_proba
from shufflenet import get_shufflenet

Using TensorFlow backend.


In [2]:
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
def SOFTMAX(s_):
    return np.exp(s_) / np.matmul(np.ones((1, s_.shape[0])), np.exp(s_))

In [4]:
def soft_targets(temp, logits, n_classes):
    soft_targets_ = np.zeros((len(logits), n_classes))
    for i in range(len(logits)):
        soft_targets_[i] = SOFTMAX(logits[i]/temp)
    return soft_targets_

In [5]:
def knowledge_distillation_loss(y_true, y_pred, alpha):

    # Extract the one-hot encoded values and the softs separately so that we can create two objective functions
    y_true, y_true_softs = y_true[: , :nb_classes], y_true[: , nb_classes:]
    
    y_pred, y_pred_softs = y_pred[: , :nb_classes], y_pred[: , nb_classes:]
    
    loss =(alpha * tf.keras.losses.categorical_crossentropy(y_true, y_pred) +
           tf.keras.losses.categorical_crossentropy(y_true_softs, y_pred_softs))
    return loss

def acc(y_true, y_pred):
    y_true = y_true[:, :nb_classes]
    y_pred = y_pred[:, :nb_classes]
    return tf.keras.metrics.categorical_accuracy(y_true, y_pred)

In [6]:
# this file is created after training is finished
config_path = 'logs/run0/model_config.txt'

# folder where validation dataset is
validation_images = 'tiny-imagenet-200/validation/'

# this file is created when you run `image_dataset_to_tfrecords.py`
class_encoder_path = 'tiny-imagenet-200/class_encoder.npy'

# this file comes with dataset
class_names_file = 'tiny-imagenet-200/words.txt'

In [7]:
# folder name -> class name in human readable format
class_names = pd.read_csv(class_names_file, sep='\t', header=None)
names = dict(class_names.set_index(0)[1])

# folder name -> class index
encoder = np.load(class_encoder_path)[()]

# class index -> class name in human readable format
decoder = {encoder[i]: names[i] for i in encoder}

In [8]:
teacher_train_logits = np.load(
    'new_teacher_train_logits.npz')['arr_0']
teacher_val_logits = np.load(
    'new_teacher_val_logits.npz')['arr_0']

In [9]:
x_val = np.load('x_val.npz')['arr_0']
x_train = np.load('new_x_train.npz')['arr_0']

In [10]:
y_train = np.load('new_y_train.npz')['arr_0']
y_val = np.load('y_val.npz')['arr_0']

In [ ]:
#Let's verify the teacher predictions on the validation set  are the same as before:

In [11]:
config_path = 'logs/run0/model_config.txt'
config = json.load(open(config_path))

graph, ops = get_shufflenet(
    groups=config['groups'], 
    complexity_scale_factor=config['complexity_scale_factor']
)

In [12]:
predictions_val = predict_proba(graph, ops, x_val, run=config['run'])
predictions_argmax = np.argmax(predictions_val, 1)

In [13]:
np.mean((y_val == predictions_argmax))

0.5072213181448332

In [ ]:
#Let's verify the teacher predictions on the training set  are the same as before:

In [15]:
#I'm batching so my computer doesn't pass out.
batch_size = 10000
# predictions_training = [predict_proba(graph, ops, X[i], run=config['run']) for i in ]
predictions_training = []
for batch_idx in range(x_train.shape[0] // batch_size + 1):
    #print("new batch")
    X_batch = x_train[batch_idx * batch_size: (batch_idx + 1) * batch_size]
    batch_pred = predict_proba(graph, ops, X_batch, run=config['run'])
    predictions_training.append(batch_pred)
concatenated_predictions_training = predictions_training[0]
for batch_prediction in predictions_training[1:]:
    concatenated_predictions_training = np.concatenate((concatenated_predictions_training, batch_prediction), axis=0)


In [17]:
predictions_argmax = np.argmax(concatenated_predictions_training, 1)

In [18]:
np.mean((y_train == predictions_argmax))

0.598620886340256

In [15]:
# encoding labels in one hot

In [14]:
nb_classes = 200
#one hot representation
y_train = to_categorical(y_train)
y_val= to_categorical(y_val)

In [ ]:
#Training without distillation:

In [16]:
#model without distillation:
model = Sequential()
model.add(Flatten(input_shape=(56,56,3)))
model.add(Dense(800, activation='relu'))
model.add(Dense(800, activation='relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tensor_board = TensorBoard('./logs/tiny-imagenet-t')

In [17]:
model.fit(x_train, y_train, batch_size=256, epochs=50, verbose=1,
          validation_data=(x_val, y_val), callbacks=[tensor_board])

Train on 98179 samples, validate on 9832 samples
Epoch 1/50
98179/98179 [==============================] - 9s 91us/step - loss: 5.0301 - acc: 0.0321 - val_loss: 4.7585 - val_acc: 0.0504
Epoch 2/50
98179/98179 [==============================] - 9s 89us/step - loss: 4.6713 - acc: 0.0603 - val_loss: 4.6312 - val_acc: 0.0661
Epoch 3/50
98179/98179 [==============================] - 9s 89us/step - loss: 4.5427 - acc: 0.0765 - val_loss: 4.5402 - val_acc: 0.0783
Epoch 4/50
98179/98179 [==============================] - 9s 89us/step - loss: 4.4489 - acc: 0.0870 - val_loss: 4.4932 - val_acc: 0.0878
Epoch 5/50
98179/98179 [==============================] - 9s 89us/step - loss: 4.3699 - acc: 0.0968 - val_loss: 4.4909 - val_acc: 0.0831
Epoch 6/50
98179/98179 [==============================] - 9s 89us/step - loss: 4.2952 - acc: 0.1063 - val_loss: 4.4494 - val_acc: 0.0933
Epoch 7/50
98179/98179 [==============================] - 9s 90us/step - loss: 4.2334 - acc: 0.1143 - val_loss: 4.4539 - val_acc:

In [32]:
#Training with distillation:

In [18]:
#soft targets
temp = 20
nb_classes = 200
train_soft_targets = soft_targets(temp, teacher_train_logits, nb_classes)
val_soft_targets = soft_targets(temp, teacher_val_logits, nb_classes)

In [20]:
Y_train_new = np.concatenate([y_train, train_soft_targets], axis=1)
Y_val_new =  np.concatenate([y_val, val_soft_targets], axis =1)

In [23]:
loss_weight = 0.5 * 1 / temp**2

In [24]:
loss_weight

0.00125

In [25]:
1 / 800

0.00125

In [39]:
temp=4
#student model:
nb_classes = 200
student_m = Sequential()
student_m.add(Flatten(input_shape=(56,56,3)))
student_m.add(Dense(800, activation='relu'))
student_m.add(Dense(800, activation='relu'))
student_m.add(Dense(nb_classes))
#student_m.add(Activation('softmax'))
student_m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(student_m.summary())

logits = student_m.layers[-1].output
probs = Activation('softmax')(logits)

logits_T = Lambda(lambda x: x / temp)(logits)
probs_T = Activation('softmax')(logits_T)

output = concatenate([probs, probs_T])

student_m = Model(student_m.input, output)

loss_weight = 0.5 * 1 / temp**2
student_m.compile(optimizer='adam',
                      loss=lambda y_true, y_pred: knowledge_distillation_loss(y_true, y_pred, loss_weight),
                      metrics=[acc])

In [33]:
student_m.fit(x_train, Y_train_new,
              batch_size=256,
              epochs=100,
              verbose=1,
              validation_data= (x_val, Y_val_new))

Train on 98179 samples, validate on 9832 samples
Epoch 1/100
98179/98179 [==============================] - 10s 99us/step - loss: 5.3033 - acc: 0.0373 - val_loss: 5.3029 - val_acc: 0.0548
Epoch 2/100
98179/98179 [==============================] - 9s 88us/step - loss: 5.3026 - acc: 0.0674 - val_loss: 5.3025 - val_acc: 0.0747
Epoch 3/100
98179/98179 [==============================] - 9s 89us/step - loss: 5.3023 - acc: 0.0858 - val_loss: 5.3023 - val_acc: 0.0855
Epoch 4/100
98179/98179 [==============================] - 9s 88us/step - loss: 5.3021 - acc: 0.0987 - val_loss: 5.3022 - val_acc: 0.0928
Epoch 5/100
98179/98179 [==============================] - 9s 88us/step - loss: 5.3019 - acc: 0.1090 - val_loss: 5.3022 - val_acc: 0.0962
Epoch 6/100
98179/98179 [==============================] - 9s 88us/step - loss: 5.3018 - acc: 0.1179 - val_loss: 5.3021 - val_acc: 0.1021
Epoch 7/100
98179/98179 [==============================] - 9s 89us/step - loss: 5.3016 - acc: 0.1266 - val_loss: 5.3020 - 

In [23]:
student_m.fit(x_train, Y_train_new,
              batch_size=256,
              epochs=50,
              verbose=1,
              validation_data= (x_val, Y_val_new))

Train on 98179 samples, validate on 9832 samples
Epoch 1/50
98179/98179 [==============================] - 8s 82us/step - loss: 10.5376 - acc: 0.0152 - val_loss: 10.4530 - val_acc: 0.0235
Epoch 2/50
98179/98179 [==============================] - 7s 71us/step - loss: 10.3702 - acc: 0.0283 - val_loss: 10.3034 - val_acc: 0.0328
Epoch 3/50
98179/98179 [==============================] - 6s 65us/step - loss: 10.2514 - acc: 0.0381 - val_loss: 10.2276 - val_acc: 0.0374
Epoch 4/50
98179/98179 [==============================] - 7s 69us/step - loss: 10.1776 - acc: 0.0463 - val_loss: 10.1702 - val_acc: 0.0451
Epoch 5/50
98179/98179 [==============================] - 7s 70us/step - loss: 10.1177 - acc: 0.0531 - val_loss: 10.1226 - val_acc: 0.0513
Epoch 6/50
98179/98179 [==============================] - 7s 70us/step - loss: 10.0665 - acc: 0.0585 - val_loss: 10.0894 - val_acc: 0.0561
Epoch 7/50
98179/98179 [==============================] - 7s 69us/step - loss: 10.0235 - acc: 0.0636 - val_loss: 10.0